In [9]:
import pandas as pd

In [10]:
path = r"C:\Users\trrallele\Momentum Metropolitan\REALEARN\CRYPTO STRATEGIES IN PYTHON\GPT\kracken-gpt\l2_data_logs\XBT-USD_orderbook_2025-05-10.csv"

In [11]:
df = pd.read_csv(path)

In [20]:
df.columns


Index(['2025-05-10 19:04:17', 'bid', '103193.5', '3.29568168'], dtype='object')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your snapshot CSV log
df = pd.read_csv(path, parse_dates=['timestamp'])

# Preprocessing: calculate mid price
df['mid_price'] = (df['best_bid'] + df['best_ask']) / 2

# Parameters
wall_threshold = 100  # Customize this based on your volume units
proximity_ticks = 5   # Price levels away from mid to count as "nearby"

signals = []

# Loop over snapshots
for i, row in df.iterrows():
    bid_levels = eval(row['bids'])  # Assuming bids stored like [(price, volume), ...]
    ask_levels = eval(row['asks'])

    mid = row['mid_price']

    # Find largest wall near mid price
    nearby_bids = [(p, v) for p, v in bid_levels if p >= mid - proximity_ticks]
    nearby_asks = [(p, v) for p, v in ask_levels if p <= mid + proximity_ticks]

    largest_bid = max(nearby_bids, key=lambda x: x[1], default=(None, 0))
    largest_ask = max(nearby_asks, key=lambda x: x[1], default=(None, 0))

    # Simple signal rules
    if largest_bid[1] > wall_threshold:
        signals.append((row['timestamp'], 'LONG', mid))
    elif largest_ask[1] > wall_threshold:
        signals.append((row['timestamp'], 'SHORT', mid))

# Convert signals to DataFrame
signal_df = pd.DataFrame(signals, columns=['timestamp', 'signal', 'price'])

# Plotting
plt.figure(figsize=(12,6))
plt.plot(df['timestamp'], df['mid_price'], label='Mid Price', color='black')

# Plot signals
longs = signal_df[signal_df['signal'] == 'LONG']
shorts = signal_df[signal_df['signal'] == 'SHORT']
plt.scatter(longs['timestamp'], longs['price'], color='green', label='Long Signal', marker='^')
plt.scatter(shorts['timestamp'], shorts['price'], color='red', label='Short Signal', marker='v')

plt.legend()
plt.title("Liquidity Wall-Based Trade Signals")
plt.xlabel("Time")
plt.ylabel("Price")
plt.grid(True)
plt.show()


KeyError: 'best_bid'